In [1]:
import pandas as pd
import numpy as np
import time
import os
import glob
import json
from collections import defaultdict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

## Configuration

In [6]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../../results'
# ID of the training and test data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0005_19042021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

pd.set_option('display.max_columns', None)

Model trained with data: 0005_19042021


## Model

In [7]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

print('Train -> X: {}, Y: {}'.format(X_train.shape, Y_train.shape))
print('Test -> X: {}, Y: {}'.format(X_test.shape, Y_test.shape))

results_ls = []

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

params = {
    'objective': 'reg:squarederror',
    'booster': 'gbtree',
}


eval_set = [(X_test, Y_test)]
# Setup the model
model = MultiOutputRegressor(estimator=xgb.XGBRegressor(**params, nthread=1, seed=0), n_jobs=-1)

t_start = time.time()
model.fit(X_train, Y_train, eval_metric='rmse', eval_set=eval_set, early_stopping_rounds=5, verbose=True)
tr_time = time.time() - t_start
print('Training time: {:.4f}'.format(tr_time))

train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

results = {
    'Train': {
        'MAE': mean_absolute_error(Y_train, train_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_train, train_preds, multioutput='raw_values'),
        'R2': r2_score(Y_train, train_preds, multioutput='raw_values')
    },
    'Test': {
        'MAE': mean_absolute_error(Y_test, test_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_test, test_preds, multioutput='raw_values'),
        'R2': r2_score(Y_test, test_preds, multioutput='raw_values')
    }       
    
}

results_summary = {}
results_summary['Training time'] = tr_time
# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))
            results_summary[' '.join([subset, force, loss])] = '{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores))
            
results_ls.append(results_summary)

In [9]:
params = {
    'objective': 'reg:squarederror',
    'booster': 'gbtree', 
    'eta': 0.4, 
    'gamma': 0.5, 
    'max_depth': 100,
    'subsample': 0.75,
    'lambda': 0.8,
    'nthread': 8,
    'seed': 0
}

results = defaultdict(list)

tr_time = []
for target in range(Y_train.shape[1]):

    dtrain = xgb.DMatrix(data=X_train, label=Y_train[:, target])
    dtest = xgb.DMatrix(data=X_test, label=Y_test[:, target])

    callbacks = [xgb.callback.EarlyStopping(rounds=5, metric_name='rmse', maximize=False, save_best=True)]
    
    t_start = time.time()
    model = xgb.train(params, dtrain, evals=[(dtest, 'rmse')], callbacks=callbacks, verbose_eval=False)
    tr_time.append(time.time() - t_start)
    
    train_preds = model.predict(dtrain)
    test_preds = model.predict(dtest)

    results['Train_MAE'].append(mean_absolute_error(Y_train[:, target], train_preds))
    results['Train_MSE'].append(mean_squared_error(Y_train[:, target], train_preds))
    results['Train_R2'].append(r2_score(Y_train[:, target], train_preds))
    results['Test_MAE'].append(mean_absolute_error(Y_test[:, target], test_preds))
    results['Test_MSE'].append(mean_squared_error(Y_test[:, target], test_preds))
    results['Test_R2'].append(r2_score(Y_test[:, target], test_preds))
    
print('Training time: {:.4f}'.format(sum(tr_time)))

results_summary = {}
results_summary['Training time'] = sum(tr_time)
# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results['_'.join([subset, loss])][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))
            results_summary[' '.join([subset, force, loss])] = '{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores))
            
results_ls.append(results_summary)

Training time: 1877.5659
Train Fx MAE: 4.0990 ± 1.7510
Train Fx MSE: 35.1781 ± 25.4498
Train Fx R2: 0.8650 ± 0.1237
Train Fy MAE: 4.6105 ± 4.5224
Train Fy MSE: 78.9545 ± 127.4982
Train Fy R2: 0.7493 ± 0.1970
Train Fz MAE: 1.7381 ± 1.4834
Train Fz MSE: 9.9064 ± 13.5909
Train Fz R2: 0.9542 ± 0.0884
Test Fx MAE: 11.4675 ± 3.8314
Test Fx MSE: 294.2906 ± 185.8058
Test Fx R2: 0.0768 ± 0.2103
Test Fy MAE: 10.6508 ± 6.7819
Test Fy MSE: 358.9070 ± 427.3204
Test Fy R2: -0.0967 ± 0.2462
Test Fz MAE: 17.1793 ± 7.6722
Test Fz MSE: 762.1315 ± 785.6772
Test Fz R2: 0.0677 ± 0.1618


In [10]:
pd.DataFrame(results_ls)

,Training time,Train Fx MAE,Train Fx MSE,Train Fx R2,Train Fy MAE,Train Fy MSE,Train Fy R2,Train Fz MAE,Train Fz MSE,Train Fz R2,Test Fx MAE,Test Fx MSE,Test Fx R2,Test Fy MAE,Test Fy MSE,Test Fy R2,Test Fz MAE,Test Fz MSE,Test Fz R2
0,276.241745,7.5300 ± 2.3571,118.1126 ± 71.1675,0.5924 ± 0.1029,7.0672 ± 3.9927,127.8950 ± 122.4373,0.4428 ± 0.1501,10.6118 ± 3.3114,242.3457 ± 169.5071,0.6278 ± 0.0980,11.0678 ± 3.8568,262.9086 ± 174.7427,0.1960 ± 0.1819,9.8929 ± 6.5332,318.4010 ± 390.0672,0.0797 ± 0.1680,16.7121 ± 7.3173,664.4588 ± 658.7176,0.1602 ± 0.1679
1,1877.565883,4.0990 ± 1.7510,35.1781 ± 25.4498,0.8650 ± 0.1237,4.6105 ± 4.5224,78.9545 ± 127.4982,0.7493 ± 0.1970,1.7381 ± 1.4834,9.9064 ± 13.5909,0.9542 ± 0.0884,11.4675 ± 3.8314,294.2906 ± 185.8058,0.0768 ± 0.2103,10.6508 ± 6.7819,358.9070 ± 427.3204,-0.0967 ± 0.2462,17.1793 ± 7.6722,762.1315 ± 785.6772,0.0677 ± 0.1618
